<a href="https://colab.research.google.com/github/dljaimes/AdvancedMethodsinDataAnalysis1Grupo_5/blob/main/Proyecto_3_Leo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 3


# Movie Genre Classification

Classify a movie genre based on its plot.

<img src="moviegenre.png"
     style="float: left; margin-right: 10px;" />




https://www.kaggle.com/c/miia4201-202019-p3-moviegenreclassification/overview

### Data

Input:
- movie plot

Output:
- Probability of the movie belonging to each genre


### Evaluation

- 50% Report with all the details of the solution, the analysis and the conclusions. The report cannot exceed 10 pages, must be send in PDF format and must be self-contained.
- 50% Performance in the Kaggle competition (The grade for each group will be proportional to the ranking it occupies in the competition. The group in the first place will obtain 5 points, for each position below, 0.25 points will be subtracted, that is: first place: 5 points, second: 4.75 points, third place: 4.50 points ... eleventh place: 2.50 points, twelfth place: 2.25 points).


### Deadline
- The project must be carried out in the groups assigned.
- Use clear and rigorous procedures.
- The delivery of the project is on August 1st, 2021, 11:59 pm, through Bloque Neón.
- No projects will be received after the delivery time or by any other means than the one established. 




### Acknowledgements

We thank Professor Fabio Gonzalez, Ph.D. and his student John Arevalo for providing this dataset.

See https://arxiv.org/abs/1702.01992

## Sample Submission

In [ ]:
!pip install -q wordcloud
!pip install livelossplot==0.1.2

In [24]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from livelossplot import PlotLossesKeras

import keras
from keras import backend as K
from sklearn.model_selection import train_test_split
import re
import nltk

from nltk.corpus import stopwords
from livelossplot import PlotLossesKeras
import warnings
warnings.filterwarnings("ignore")

In [16]:
# Se descargan los Stopwords:
nltk.download('stopwords')
nltk.corpus.stopwords.words('english')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [148]:
dataTraining = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/raw/master/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [19]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [20]:
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


# PREPROCESAMIENTO

In [149]:
# Se eliminan caracteres especiales:

import re
def pre_process(text):
    # lowercase
    text = text.lower()
    # tags
    text = re.sub('&lt;/?.*?&gt;',' &lt;&gt; ',text)
    # special characters and digits
    text=re.sub('(\\d|\\W)+',' ',text)
    # remove punctuation
    #text = re.sub('[.;:!\'?,\"()\[\]]', '', text)
    #text = [REPLACE.sub('', line) for line in text]
    
    return text
dataTraining['plot_low']=dataTraining['plot'].apply(lambda x:pre_process(x))
dataTesting['plot_low']=dataTesting['plot'].apply(lambda x:pre_process(x))


import nltk
nltk.corpus.stopwords.words('english')
nltk.download('wordnet') 
from nltk.corpus import stopwords
###############################################################################

# se eliminan stopwords

english_stop_words=stopwords.words('english')

def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

dataTraining['plot_low_rm'] = remove_stop_words(dataTraining['plot_low'])
dataTesting['plot_low_rm'] = remove_stop_words(dataTesting['plot_low'])

# Stemming:

from nltk.stem.porter import PorterStemmer
def get_stemmed_text(corpus):
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

dataTraining['plot_low_rm_stem'] = get_stemmed_text(dataTraining['plot_low_rm'])
dataTesting['plot_low_rm_stem'] = get_stemmed_text(dataTesting['plot_low_rm'])

# LEMATIZACION:

from nltk.stem import WordNetLemmatizer
def lemma(texto):
  lemmatizador = WordNetLemmatizer()
  return [' '.join([lemmatizador.lemmatize(word) for word in review.split()]) for review in texto]

dataTraining['plot_low_rm_lemma'] = lemma(dataTraining['plot_low_rm'])
dataTesting['plot_low_rm_lemma'] = lemma(dataTesting['plot_low_rm'])


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [150]:
dataTraining

,year,title,plot,genres,rating,plot_low,plot_low_rm,plot_low_rm_stem,plot_low_rm_lemma
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0,most is the story of a single father who takes...,story single father takes eight year old son w...,stori singl father take eight year old son wor...,story single father take eight year old son wo...
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6,a serial killer decides to teach the secrets o...,serial killer decides teach secrets satisfying...,serial killer decid teach secret satisfi caree...,serial killer decides teach secret satisfying ...
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2,in sweden a female blackmailer with a disfigur...,sweden female blackmailer disfiguring facial s...,sweden femal blackmail disfigur facial scar me...,sweden female blackmailer disfiguring facial s...
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4,in a friday afternoon in new york the presiden...,friday afternoon new york president tredway co...,friday afternoon new york presid tredway corpo...,friday afternoon new york president tredway co...
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6,in los angeles the editor of a publishing hous...,los angeles editor publishing house carol hunn...,lo angel editor publish hous carol hunnicut go...,los angeles editor publishing house carol hunn...
...,...,...,...,...,...,...,...,...,...
8417,2010,Our Family Wedding,""" our marriage , their wedding . "" it ' s l...","['Comedy', 'Romance']",4.9,our marriage their wedding it s lesson number...,marriage wedding lesson number one newly engag...,marriag wed lesson number one newli engag coup...,marriage wedding lesson number one newly engag...
1592,1984,Conan the Destroyer,"the wandering barbarian , conan , alongside ...","['Action', 'Adventure', 'Fantasy']",5.8,the wandering barbarian conan alongside his go...,wandering barbarian conan alongside goofy rogu...,wander barbarian conan alongsid goofi rogu pal...,wandering barbarian conan alongside goofy rogu...
1723,1955,Kismet,"like a tale spun by scheherazade , kismet fol...","['Adventure', 'Musical', 'Fantasy', 'Comedy', ...",6.4,like a tale spun by scheherazade kismet follow...,like tale spun scheherazade kismet follows rem...,like tale spun scheherazad kismet follow remar...,like tale spun scheherazade kismet follows rem...
7605,1982,The Secret of NIMH,"mrs . brisby , a widowed mouse , lives in a...","['Animation', 'Adventure', 'Drama', 'Family', ...",7.6,mrs brisby a widowed mouse lives in a cinder b...,mrs brisby widowed mouse lives cinder block ch...,mr brisbi widow mous live cinder block childre...,mr brisby widowed mouse life cinder block chil...


In [292]:
for i in range(1,3):
  print(i)

1
2


# COUNT VECTORIZER()


In [299]:
vect = CountVectorizer(max_features=5000)
X_dtm = vect.fit_transform(dataTraining['plot_low_rm_stem'])
print(X_dtm.shape)
voca_coun_vec = vect.vocabulary_

(7895, 5000)


In [294]:
print(voca_coun_vec)

{'stori': 4225, 'singl': 4030, 'father': 1584, 'take': 4349, 'eight': 1353, 'year': 4961, 'old': 3110, 'son': 4106, 'work': 4926, 'railroad': 3545, 'bridg': 518, 'day': 1052, 'boy': 495, 'meet': 2789, 'woman': 4918, 'board': 460, 'train': 4542, 'drug': 1295, 'abus': 15, 'goe': 1883, 'engin': 1415, 'room': 3779, 'tell': 4399, 'stay': 4200, 'edg': 1340, 'nearbi': 2985, 'lake': 2457, 'ship': 3992, 'come': 820, 'lift': 2546, 'though': 4460, 'suppos': 4306, 'arriv': 235, 'hour': 2117, 'later': 2472, 'happen': 1973, 'see': 3903, 'tri': 4566, 'warn': 4827, 'abl': 7, 'approach': 212, 'fall': 1548, 'gear': 1810, 'attempt': 267, 'lower': 2634, 'leav': 2497, 'horrif': 2105, 'choic': 714, 'crush': 1001, 'peopl': 3271, 'complet': 855, 'fact': 1537, 'die': 1172, 'save': 3852, 'addict': 50, 'look': 2601, 'window': 4899, 'movi': 2929, 'end': 1405, 'man': 2691, 'wander': 4804, 'new': 3015, 'citi': 737, 'longer': 2599, 'hold': 2075, 'small': 4067, 'babi': 302, 'run': 3802, 'parallel': 3216, 'name': 2962

In [286]:
X_dtm.todense()[0][0][0]

matrix([[0, 0, 0, ..., 0, 0, 0]])

In [283]:
df_dict =  pd.DataFrame.from_dict(voca_coun_vec,  orient='index')

In [153]:
print(vect.get_feature_names()[:50])

['abandoned', 'ability', 'able', 'accepts', 'accident', 'accidentally', 'across', 'act', 'action', 'actor', 'actress', 'actually', 'adam', 'adult', 'adventure', 'affair', 'age', 'aged', 'agent', 'ago', 'agrees', 'air', 'alan', 'alex', 'alice', 'alien', 'alive', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'america', 'american', 'among', 'ancient', 'andrew', 'andy', 'angeles', 'animal', 'ann', 'anna', 'annie', 'another', 'answer', 'anyone', 'anything', 'apart']


# Se crea un diccionario con el tokenizador de Keras

In [244]:
tok_keras = Tokenizer(num_words=1000, oov_token='oov')
tok_keras.fit_on_texts(dataTraining['plot_low_rm_lemma'])
vocabulario = tok_keras.word_index

In [245]:
print(vocabulario)

{'oov': 1, 'n': 2, 'life': 3, 'one': 4, 'find': 5, 'get': 6, 'new': 7, 'friend': 8, 'year': 9, 'man': 10, 'go': 11, 'love': 12, 'two': 13, 'family': 14, 'take': 15, 'time': 16, 'want': 17, 'father': 18, 'make': 19, 'young': 20, 'woman': 21, 'also': 22, 'day': 23, 'old': 24, 'world': 25, 'meet': 26, 'back': 27, 'wife': 28, 'way': 29, 'work': 30, 'help': 31, 'come': 32, 'try': 33, 'home': 34, 'story': 35, 'mother': 36, 'school': 37, 'son': 38, 'see': 39, 'however': 40, 'house': 41, 'daughter': 42, 'first': 43, 'girl': 44, 'town': 45, 'tell': 46, 'soon': 47, 'like': 48, 'people': 49, 'brother': 50, 'know': 51, 'may': 52, 'job': 53, 'child': 54, 'becomes': 55, 'begin': 56, 'decides': 57, 'city': 58, 'money': 59, 'film': 60, 'york': 61, 'three': 62, 'even': 63, 'end': 64, 'boy': 65, 'must': 66, 'war': 67, 'fall': 68, 'night': 69, 'men': 70, 'named': 71, 'relationship': 72, 'death': 73, 'turn': 74, 'set': 75, 'police': 76, 'best': 77, 'john': 78, 'thing': 79, 'american': 80, 'plan': 81, 'wou

In [247]:
# Se transforma cada texto en palabras de acuerdo con su diccionario de índices:

frases_num = tok_keras.texts_to_sequences(dataTraining['plot_low_rm_lemma'])
matriz_pad =  pad_sequences(frases_num, padding='post')

In [248]:
matriz_pad.shape

(7895, 820)

## SE CREA LA VARIABLE Y CON SUS MULTICLASES

In [159]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [258]:
dataTraining.head(2)

,year,title,plot,genres,rating,plot_low,plot_low_rm,plot_low_rm_stem,plot_low_rm_lemma
3107,2003,Most,most is the story of a single father who takes...,"[Short, Drama]",8.0,most is the story of a single father who takes...,story single father takes eight year old son w...,stori singl father take eight year old son wor...,story single father take eight year old son wo...
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"[Comedy, Crime, Horror]",5.6,a serial killer decides to teach the secrets o...,serial killer decides teach secrets satisfying...,serial killer decid teach secret satisfi caree...,serial killer decides teach secret satisfying ...


In [42]:
le.classes_

array(['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
       'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News',
       'Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War',
       'Western'], dtype=object)

## Particion datos de Test y Train

In [301]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm.todense(), y_genres, test_size=0.33, random_state=42)
#X_train, X_test, y_train_genres, y_test_genres = train_test_split(matriz_pad, y_genres, test_size=0.33, random_state=42)

### Train multi-class multi-label model

In [170]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier

In [171]:
nums = list(range(24))
lis_auc = []

for n in nums:
  lis_auc.append(roc_auc_score(y_test_genres[:,n], y_pred_genres[:,n]))
lis_auc = np.array(lis_auc)

In [172]:
lis_auc[17]

0.7770526115193341

## MODELO GAUSIANO NB

In [ ]:
help(GaussianNB)

In [ ]:
modelo_nb = GaussianNB

In [303]:
# Usando Naive Bayes:
nb_c = OneVsRestClassifier(GaussianNB())
#gbc = OneVsRestClassifier(GradientBoostingClassifier(max_depth=10,n_estimators=150,max_features=))

In [304]:
%%time
nb_c.fit(X_train, y_train_genres)

CPU times: user 9.02 s, sys: 107 ms, total: 9.13 s
Wall time: 9.09 s


OneVsRestClassifier(estimator=GaussianNB(priors=None, var_smoothing=1e-09),
                    n_jobs=None)

In [120]:
X_train.shape

(5289, 1000)

In [305]:
pred_gbc = nb_c.predict_proba(X_test)
roc_auc_score(y_test_genres, pred_gbc, average='macro')

0.5467577922457182

In [ ]:
pd.DataFrame(pred_gbc)

In [121]:
# Calibración Gradient Boosting:




5

In [105]:
pred_nb = nb_c.predict_proba(X_test)
roc_auc_score(y_test_genres, pred_nb, average='macro')

0.6505733721375314

In [185]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

## RANDOM FOREST

In [306]:
clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=500, max_depth=5, random_state=42))

In [307]:
%%time

clf.fit(X_train, y_train_genres);

CPU times: user 35.7 s, sys: 2.3 s, total: 38 s
Wall time: 6min 31s


OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=5,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                               

In [308]:
y_pred_genres = clf.predict_proba(X_test)
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.8238489387536768

In [ ]:
####################### resultados usando COUNTVETORIZER() y Lemma:


"""
------- Para un número de palabras = 1 mil

n_jobs=-1, n_estimators=200, max_depth=7, random_state=42 : 0.7872131570577036
n_jobs=-1, n_estimators=200, max_depth=10, random_state=42 : 0.7840666999257996
n_jobs=-1, n_estimators=250, max_depth=7, random_state=42 : 0.7901943042466396
n_jobs=-1, n_estimators=250, max_depth=7, random_state=42 : 0.7908025199296361
n_jobs=-1, n_estimators=500, max_depth=7, random_state=42 : 0.7931854788836633
n_jobs=-1, n_estimators=500, max_depth=5, random_state=42 : 0.7934482357988548
n_jobs=-1, n_estimators=700, max_depth=3, random_state=42 : 0.7926350713330667
n_jobs=-1, n_estimators=500, max_depth=3, random_state=42 : 0.7918581885906483
n_jobs=-1, n_estimators=500, max_depth=4, random_state=42 : 0.7916051690107286


------- Para un número de palabras = 3 mil

n_jobs=-1, n_estimators=500, max_depth=5, random_state=42 : 0.8105434607387648



-------- Para un número de palabras = 5 mil

n_jobs=-1, n_estimators=500, max_depth=5, random_state=42  : 0.8203351605539488

"""


################## resultados usando COUNTVETORIZER() y Stemm:
"""
-------- Para un número de palabras = 5 mil

n_jobs=-1, n_estimators=500, max_depth=5, random_state=42  : 0.8238489387536768


"""




################## resultados usando COUNTVETORIZER(ngram_range=range(1,2)) y Stemm:

"""

n_jobs=-1, n_estimators=500, max_depth=5, random_state=42  : 0.8204102576175692

"""


In [309]:
dataTesting

,year,title,plot,plot_low,plot_low_rm,plot_low_rm_stem,plot_low_rm_lemma
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ....",who meets by fate shall be sealed by fate ther...,meets fate shall sealed fate theresa osborne r...,meet fate shall seal fate theresa osborn run a...,meet fate shall sealed fate theresa osborne ru...
4,1978,Midnight Express,"the true story of billy hayes , an american c...",the true story of billy hayes an american coll...,true story billy hayes american college studen...,true stori billi hay american colleg student c...,true story billy hayes american college studen...
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...,martin vail left the chicago da s office to be...,martin vail left chicago da office become succ...,martin vail left chicago da offic becom succes...,martin vail left chicago da office become succ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...,husband and wife americans dr eugene and mrs h...,husband wife americans dr eugene mrs helen fer...,husband wife american dr eugen mr helen fergus...,husband wife american dr eugene mr helen fergu...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...,the coroner and scientist dr warren chapin is ...,coroner scientist dr warren chapin researching...,coron scientist dr warren chapin research shiv...,coroner scientist dr warren chapin researching...
...,...,...,...,...,...,...,...
11263,2008,The Fifth Commandment,"in bangkok , an assassin who turns down a job...",in bangkok an assassin who turns down a job th...,bangkok assassin turns job hits close home fin...,bangkok assassin turn job hit close home find ...,bangkok assassin turn job hit close home find ...
11265,2003,Coffee and Cigarettes,eleven separate vignettes are presented . in ...,eleven separate vignettes are presented in eac...,eleven separate vignettes presented celebritie...,eleven separ vignett present celebr play semi ...,eleven separate vignette presented celebrity p...
11269,1957,Pal Joey,"joey evans is charming , handsome , funny , ...",joey evans is charming handsome funny talented...,joey evans charming handsome funny talented fi...,joey evan charm handsom funni talent first cla...,joey evans charming handsome funny talented fi...
11270,2002,Jonah: A VeggieTales Movie,when the singing veggies encounter some car tr...,when the singing veggies encounter some car tr...,singing veggies encounter car trouble stranded...,sing veggi encount car troubl strand old rundo...,singing veggie encounter car trouble stranded ...


In [ ]:
X_dtm_test =  vect.transform(dataTesting['plot_low_rm_stem'])
X_dtm_test.todense()

In [314]:
# Predicciones en la base de Testing:



pred_test_rf = clf.predict_proba(X_dtm_test)


In [319]:
pred_test_rf = pd.DataFrame(pred_test_rf)
pred_test_rf.columns = ['p_'+i for i in le.classes_]

In [332]:
pred_test_rf.index= dataTesting.index

In [333]:
pred_test_rf

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,p_Film-Noir,p_History,p_Horror,p_Music,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.141583,0.115009,0.028506,0.043182,0.376818,0.157969,0.048998,0.505890,0.077178,0.094007,0.017428,0.031782,0.106200,0.041758,0.032876,0.084171,0.000186,0.278037,0.076081,0.009882,0.026576,0.230310,0.035176,0.023568
4,0.149209,0.113363,0.030664,0.046575,0.366637,0.178097,0.050001,0.504799,0.074665,0.077250,0.017195,0.031951,0.106566,0.033964,0.030071,0.082463,0.005369,0.202292,0.073574,0.011243,0.025091,0.235205,0.040948,0.023341
5,0.167899,0.122656,0.028889,0.050039,0.332644,0.333803,0.057289,0.584189,0.078742,0.089177,0.045740,0.035261,0.148510,0.034418,0.042467,0.159622,0.000250,0.262737,0.091183,0.011874,0.028783,0.344594,0.042509,0.026823
6,0.166278,0.118044,0.026254,0.045981,0.368693,0.161255,0.047223,0.535590,0.077269,0.077761,0.021804,0.049048,0.129837,0.034305,0.042086,0.105913,0.002214,0.237251,0.089256,0.009493,0.026480,0.271851,0.060772,0.023660
7,0.169209,0.137551,0.027851,0.046109,0.362413,0.194019,0.046747,0.444038,0.081763,0.112000,0.023771,0.032529,0.200336,0.035084,0.028497,0.098826,0.000307,0.207476,0.201835,0.012105,0.024893,0.247783,0.035176,0.023157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11263,0.192987,0.112609,0.026413,0.040929,0.380138,0.172888,0.046564,0.491132,0.073648,0.077584,0.016930,0.032287,0.106251,0.033964,0.028556,0.085016,0.000262,0.209321,0.073479,0.010117,0.025005,0.264768,0.035176,0.023115
11265,0.151588,0.132770,0.026436,0.042134,0.383479,0.176616,0.057504,0.493095,0.074581,0.083088,0.016944,0.038922,0.109360,0.079123,0.042776,0.093934,0.003134,0.227267,0.108263,0.014492,0.026727,0.247984,0.044823,0.023115
11269,0.148833,0.115097,0.028378,0.041109,0.397192,0.161989,0.052025,0.506427,0.079858,0.078452,0.021400,0.032709,0.107077,0.042328,0.043645,0.102121,0.000256,0.261627,0.075121,0.009580,0.026102,0.233643,0.036317,0.023063
11270,0.173939,0.246733,0.028673,0.045829,0.446083,0.157372,0.046507,0.488328,0.111202,0.082157,0.016930,0.032688,0.112642,0.042833,0.044981,0.084582,0.000461,0.251938,0.076915,0.010282,0.025587,0.229559,0.041316,0.024481


In [334]:
pred_test_rf.to_csv('pred_test_rf_5mil_dep5_nst500_counvec_stemm.csv')

In [323]:
pred_test_rf['p_Action'].describe()

count    3383.000000
mean        0.165353
std         0.026740
min         0.126125
25%         0.147677
50%         0.156833
75%         0.173368
max         0.357981
Name: p_Action, dtype: float64

## RED NEURONAL - LSTM

In [336]:
dataTraining.head(3)

,year,title,plot,genres,rating,plot_low,plot_low_rm,plot_low_rm_stem,plot_low_rm_lemma
3107,2003,Most,most is the story of a single father who takes...,"[Short, Drama]",8.0,most is the story of a single father who takes...,story single father takes eight year old son w...,stori singl father take eight year old son wor...,story single father take eight year old son wo...
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"[Comedy, Crime, Horror]",5.6,a serial killer decides to teach the secrets o...,serial killer decides teach secrets satisfying...,serial killer decid teach secret satisfi caree...,serial killer decides teach secret satisfying ...
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","[Drama, Film-Noir, Thriller]",7.2,in sweden a female blackmailer with a disfigur...,sweden female blackmailer disfiguring facial s...,sweden femal blackmail disfigur facial scar me...,sweden female blackmailer disfiguring facial s...


In [363]:
x = dataTraining['plot_low_rm_stem'].tolist()

In [364]:
x[0]

'stori singl father take eight year old son work railroad drawbridg bridg tender day boy meet woman board train drug abus bridg father goe engin room tell son stay edg nearbi lake ship come bridg lift though suppos arriv hour later train happen arriv son see tri warn father abl see oncom train approach son fall drawbridg gear work attempt lower bridg leav father horrif choic father lower bridg gear crush boy peopl train complet oblivi fact boy die tri save drug addict woman happen look train window movi end man wander new citi meet woman longer drug addict hold small babi relev narr run parallel name one femal drug addict meet climax tumultu film'

In [373]:
print(vect.vocabulary_)

{'stori': 4275, 'singl': 4076, 'father': 1643, 'take': 4412, 'eight': 1419, 'year': 4980, 'old': 3136, 'son': 4160, 'work': 4949, 'railroad': 3578, 'bridg': 542, 'day': 1122, 'boy': 514, 'meet': 2822, 'woman': 4941, 'board': 479, 'train': 4588, 'drug': 1359, 'abus': 16, 'goe': 1895, 'engin': 1484, 'room': 3824, 'tell': 4450, 'stay': 4250, 'edg': 1405, 'nearbi': 3010, 'lake': 2516, 'ship': 4027, 'come': 879, 'lift': 2606, 'though': 4504, 'suppos': 4364, 'arriv': 245, 'hour': 2152, 'later': 2532, 'happen': 2002, 'see': 3947, 'tri': 4613, 'warn': 4852, 'abl': 8, 'approach': 216, 'fall': 1618, 'gear': 1851, 'attempt': 282, 'lower': 2675, 'leav': 2561, 'horrif': 2142, 'choic': 760, 'crush': 1063, 'peopl': 3287, 'complet': 906, 'fact': 1607, 'die': 1232, 'save': 3902, 'addict': 48, 'look': 2651, 'window': 4923, 'movi': 2958, 'end': 1476, 'man': 2728, 'wander': 4843, 'new': 3043, 'citi': 786, 'longer': 2649, 'hold': 2111, 'small': 4116, 'babi': 318, 'run': 3848, 'parallel': 3227, 'name': 2987

In [372]:
plotw = ['the plot was afuel', ' I do not know what to do']

set(' '.join(plotw[0].split()))

{' ', 'a', 'e', 'f', 'h', 'l', 'o', 'p', 's', 't', 'u', 'w'}

In [358]:
# For vocabulary only the intersec characters is used to avoid issues with data collection
voc = set(''.join(x.str.split()))
vocabulary = {x: idx + 1 for idx, x in enumerate(set(voc))}

SyntaxError: ignored

In [355]:
print(vocabulary)

{'í': 1, 'ü': 2, 'ʼ': 3, 'n': 4, 'ø': 5, 'f': 6, 'j': 7, 'û': 8, 'ï': 9, 'a': 10, 'm': 11, 'v': 12, 'k': 13, 'â': 14, 'ä': 15, 'ó': 16, 'à': 17, 'e': 18, 'g': 19, 'b': 20, 'd': 21, 'c': 22, 'r': 23, 'á': 24, 'z': 25, 'p': 26, 'ê': 27, 'ñ': 28, 'l': 29, '½': 30, 'ô': 31, 'y': 32, 'w': 33, 's': 34, 'ç': 35, 't': 36, ' ': 37, 'q': 38, 'è': 39, 'ò': 40, 'ù': 41, '¹': 42, 'å': 43, 'ú': 44, 'o': 45, 'u': 46, 'x': 47, 'ë': 48, 'i': 49, 'é': 50, 'h': 51, 'ö': 52}


In [356]:
print(voc)

{'í', 'ü', 'ʼ', 'n', 'ø', 'f', 'j', 'û', 'ï', 'a', 'm', 'v', 'k', 'â', 'ä', 'ó', 'à', 'e', 'g', 'b', 'd', 'c', 'r', 'á', 'z', 'p', 'ê', 'ñ', 'l', '½', 'ô', 'y', 'w', 's', 'ç', 't', ' ', 'q', 'è', 'ò', 'ù', '¹', 'å', 'ú', 'o', 'u', 'x', 'ë', 'i', 'é', 'h', 'ö'}


In [208]:
y_pred_genres

array([[0.1314722 , 0.10144398, 0.02429839, ..., 0.22277413, 0.02714646,
        0.02071784],
       [0.14627235, 0.20968132, 0.02589153, ..., 0.21517307, 0.16379737,
        0.02126833],
       [0.20813667, 0.14195672, 0.0251613 , ..., 0.28097009, 0.03528771,
        0.02084947],
       ...,
       [0.16600292, 0.12280634, 0.02667076, ..., 0.23666575, 0.03020415,
        0.02059525],
       [0.13308577, 0.10211284, 0.02510587, ..., 0.27611282, 0.02633235,
        0.02005324],
       [0.15521158, 0.13373254, 0.02485927, ..., 0.31143636, 0.02644328,
        0.02127948]])

In [165]:
fa = pd.DataFrame(y_test_genres)
fa.columns = le.classes_
fa.sum().sort_values()

News              3
Short            30
Film-Noir        57
History          80
Western          85
Sport            94
Musical          97
Animation        99
War             102
Music           123
Documentary     129
Biography       130
Mystery         242
Fantasy         243
Sci-Fi          249
Family          252
Horror          300
Adventure       340
Action          423
Crime           468
Romance         633
Thriller        660
Comedy         1028
Drama          1283
dtype: int64

In [82]:
y_pred_genres[0].round(2)

array([0.12, 0.09, 0.02, 0.04, 0.33, 0.16, 0.19, 0.5 , 0.08, 0.06, 0.01,
       0.03, 0.1 , 0.03, 0.02, 0.06, 0.  , 0.16, 0.06, 0.02, 0.02, 0.21,
       0.02, 0.02])

In [180]:
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.7816365121286554

### Predict the testing dataset

In [64]:
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = clf.predict_proba(X_test_dtm)


In [65]:
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)

In [66]:
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,p_Film-Noir,p_History,p_Horror,p_Music,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.143030,0.101960,0.024454,0.029938,0.354552,0.138830,0.030787,0.490140,0.073159,0.101339,0.011415,0.026011,0.090889,0.084164,0.025069,0.063208,0.000000,0.362818,0.056648,0.008970,0.017522,0.202605,0.033989,0.018117
4,0.122624,0.085786,0.024213,0.084795,0.370949,0.216657,0.080359,0.515684,0.062976,0.067019,0.010105,0.031995,0.089059,0.034723,0.024734,0.060935,0.000477,0.149703,0.058190,0.014248,0.020099,0.204794,0.030438,0.018506
5,0.151364,0.110284,0.013762,0.075334,0.304837,0.448736,0.021010,0.611544,0.081741,0.169121,0.122455,0.040951,0.151860,0.031473,0.044538,0.261372,0.000000,0.335987,0.128505,0.001016,0.048658,0.423242,0.052693,0.025351
6,0.154448,0.125772,0.020991,0.064124,0.340779,0.140892,0.009133,0.632038,0.068287,0.063631,0.023088,0.041974,0.126732,0.047288,0.131074,0.088418,0.000000,0.197224,0.132208,0.001432,0.039743,0.269385,0.077607,0.017862
7,0.175143,0.210069,0.035476,0.032505,0.313850,0.243150,0.021793,0.427885,0.079781,0.143879,0.025481,0.025040,0.276873,0.023089,0.023859,0.090359,0.000048,0.205117,0.241663,0.002634,0.018403,0.259465,0.021569,0.017585


In [ ]:
res.to_csv('pred_genres_text_RF.csv', index_label='ID')